# Welkom bij Tutorial 4

Dit is alweer de laatste tutorial van Algoritmiek en Datastructuren, waarin wij **grafen** gaan bestuderen. Een graaf is een datastructuur waarin nodes (die in de context van grafen ook wel **vertices** heten) aan elkaar verbonden zijn door middel van **edges**. 

De manier waarop er meestal over grafen geschreven wordt is al een tuple $G = (V, E)$ waarbij G de graaf is, V de *set* van nodes/vertices en E de *set* van edges. Ik schrijf hier *set* schuingedrukt, omdat dit een bekende datastructuur voor ons is: een container waarin geen dubbelen mogen zitten. 

Vertices zijn uiteraard een classes die data kunnen bevatten, maar ook de edges kunnen soms data bevatten. Bijvoorbeeld, in een shortest path problem (zie Reader) stellen de vertices/nodes locaties voor, en bevatten dus de naam van de locatie, en de edges verbindingen tussen deze locaties. De edges bevatten de *afstand* (bijvoorbeeld in kilometers) tussen deze locaties. 

Grafen zijn een zeer flexibele manier van data structuren: zolang er maar verbindingen tussen nodes en edges gelegd kunnen worden is het een valide implementatie van een graaf. Dit kan je natuurlijk op heel veel verschillende manieren doen. Laten we daar eens een paar van gaan bekijken. 

We gaan dit graafje maken: https://en.wikipedia.org/wiki/File:Dijkstra_Animation.gif 

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import time
import statistics as stats
        

In [1]:
# Een zeer generieke manier om een graaf de implementeren is er 
# daarwerkelijk twee sets van te maken op basis van twee classes: 
class Vertex: 
    def __init__(self, identifier, data_):
        self.id = identifier
        self.data = data_
        
    def __eq__(self, other): #nodig om aan een set toe te voegen
        return self.id == other.id
    
    def __hash__(self): #nodig om aan een set toe te voegen
        return hash(self.id)
    
    def __repr__(self):
        return str(self.id)+":"+str(self.data)
        
class Edge:
    def __init__(self, vertex1, vertex2, data_):
        if(vertex1.id<vertex2.id):
            self.v1 = vertex1
            self.v2 = vertex2
        else:
            self.v1 = vertex2
            self.v2 = vertex1
        self.data = data_
        
    def __eq__(self, other): #nodig om aan een set toe te voegen
        return self.v1.id == other.v1.id and self.v2.id == self.v2.id
    
    def __hash__(self): #nodig om aan een set toe te voegen
        return hash(str(self.v1.id)+","+str(self.v2.id))
    
    def __repr__(self):
        return "("+str(self.v1.id)+","+str(self.v2.id)+"):"+str(self.data)

class CGraph:
    def __init__(self):
        self.V = set()
        self.E = set()
        
    def __str__(self):
        return "V: "+str(self.V)+"\nE: "+str(self.E)
            
        
# So, for a simple shortest path problem: 
gr = CGraph()
v1 = Vertex(1,"start")
v2 = Vertex(2,"")
v3 = Vertex(3,"")
v4 = Vertex(4,"")
v5 = Vertex(5,"goal")
v6 = Vertex(6,"")
gr.V.add(v1)
gr.V.add(v2)
gr.V.add(v3)
gr.V.add(v4)
gr.V.add(v5)
gr.V.add(v6)
e1 = Edge(v1,v2,7)
e2 = Edge(v1,v3,9)
e3 = Edge(v1,v6,14)
e4 = Edge(v2,v3,10)
e5 = Edge(v2,v4,15)
e6 = Edge(v3,v4,11)
e7 = Edge(v3,v6,2)
e8 = Edge(v6,v5,9)
e9 = Edge(v4,v5,6)
gr.E.add(e1)
gr.E.add(e2)
gr.E.add(e3)
gr.E.add(e4)
gr.E.add(e5)
gr.E.add(e6)
gr.E.add(e7)
gr.E.add(e8)
gr.E.add(e9)
print(gr)

# Is dit eigenlijk al best een langdradige manier van doen...

V: {1:start, 2:, 3:, 4:, 5:goal, 6:}
E: {(2,4):15, (5,6):9, (4,5):6, (2,3):10, (1,3):9, (3,6):2, (1,2):7, (1,6):14, (3,4):11}


In [1]:
#Daarom kiezen mensen er vaak voor om een graaf makkelijker te respresenteren met standaard datatypes
#bijvoorbeeld: als we een dictionary maken met node identifiers als keys, en een tuple van (data, edges) als values
#met edges als een dictionary met vertex ids van de verbonden vertices als keys en de data van de edge als key
#krijgen we zoiets voor dezelfde graaf
DGraph = dict
gr2 = {1: ("start", {2:7, 3:9, 6:14}),
       2: ("", {1:7, 3:10, 4:15}), 
       3: ("", {1:9, 2:10, 4:11, 6:2}), 
       4: ("", {2:15, 3:11, 5:6}), 
       5: ("goal", {4:6, 6:9}), 
       6: ("", {1:14, 3:2, 5:9})
      }
print(gr2)
#Dat is makkelijker in te voeren... Maar je moet wel goed bijhouden wat ookalweer wat is. 
#Bovendien heb ik er nu wat redundantie ingezet (elke node bevat de edges waaraan het verbonden zit, dus elke 
# edge zit twee maal in deze datastructuur)

{1: ('start', {2: 7, 3: 9, 6: 14}), 2: ('', {1: 7, 3: 10, 4: 15}), 3: ('', {1: 9, 2: 10, 4: 11, 6: 2}), 4: ('', {2: 15, 3: 11, 5: 6}), 5: ('goal', {4: 6, 6: 9}), 6: ('', {1: 14, 3: 2, 5: 9})}


## Opgave 1: Dijkstra

Laten we eens gaan kijken hoe goed de twee mogelijke implementaties van een graaf hierboven zijn (dus die met de expliciete klasses voor Vertices en Edges, en die die de hele graaf in één dictionary gooit. 

Schrijf implementaties van Dijkstra's algoritme voor de beide implementaties van grafen (CGraph en DGraph). Vergelijk de snelheid van beide versies van Dijkstra's algoritme door beiden meerdere keren te runnen. Vergeet niet de standaardafwijkingen te berekenen. Is één implementatie beter dan de andere? 

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import time
import statistics as stats

# Een zeer generieke manier om een graaf de implementeren is er 
# daarwerkelijk twee sets van te maken op basis van twee classes: 
class Vertex: 
    def __init__(self, identifier, data_):
        self.id = identifier
        self.data = data_
        
    def __eq__(self, other): #nodig om aan een set toe te voegen
        return self.id == other.id
    
    def __hash__(self): #nodig om aan een set toe te voegen
        return hash(self.id)
    
    def __repr__(self):
        return str(self.id)+":"+str(self.data)
        
class Edge:
    def __init__(self, vertex1, vertex2, data_):
        if(vertex1.id<vertex2.id):
            self.v1 = vertex1
            self.v2 = vertex2
        else:
            self.v1 = vertex2
            self.v2 = vertex1
        self.data = data_
        
    def __eq__(self, other): #nodig om aan een set toe te voegen
        return self.v1.id == other.v1.id and self.v2.id == self.v2.id
    
    def __hash__(self): #nodig om aan een set toe te voegen
        return hash(str(self.v1.id)+","+str(self.v2.id))
    
    def __repr__(self):
        return "("+str(self.v1.id)+","+str(self.v2.id)+"):"+str(self.data)

class CGraph:
    def __init__(self):
        self.V = set()
        self.E = set()
        
    def __str__(self):
        return "V: "+str(self.V)+"\nE: "+str(self.E)
            
        
# So, for a simple shortest path problem: 
gr = CGraph()
v1 = Vertex(1,"start")
v2 = Vertex(2,"")
v3 = Vertex(3,"")
v4 = Vertex(4,"")
v5 = Vertex(5,"goal")
v6 = Vertex(6,"")
gr.V.add(v1)
gr.V.add(v2)
gr.V.add(v3)
gr.V.add(v4)
gr.V.add(v5)
gr.V.add(v6)
e1 = Edge(v1,v2,7)
e2 = Edge(v1,v3,9)
e3 = Edge(v1,v6,14)
e4 = Edge(v2,v3,10)
e5 = Edge(v2,v4,15)
e6 = Edge(v3,v4,11)
e7 = Edge(v3,v6,2)
e8 = Edge(v6,v5,9)
e9 = Edge(v4,v5,6)
gr.E.add(e1)
gr.E.add(e2)
gr.E.add(e3)
gr.E.add(e4)
gr.E.add(e5)
gr.E.add(e6)
gr.E.add(e7)
gr.E.add(e8)
gr.E.add(e9)
print("====================CGraph====================")
print(gr)
print()

def minDistSet(a):
    lowestDist = float('inf')
    vertex = None
    for val in a:
        if val.dist < lowestDist:
            lowestDist = val.dist
            vertex = val
    return vertex

def setDefaultValues(a):
    for n in a:
        n.dist = float('inf')
        n.prev = None
        n.solved = False

def CGraphFindShortestPath(graph, start, finish):
    setDefaultValues(graph.V)
    route = []
    neighbours = []
    start.dist = 0
    N = set()
    N.add(start)
    S = set()
    while len(N) != 0:
        n = minDistSet(N)
        S.add(n)
        N.remove(n)
        n.solved = True
        if n == finish:
            break
        
        #Find neighbours of current node n
        neighbours = []
        for x in graph.E:
            if n == x.v1:
                neighbours.append((x.v2, x)) #(v1, v2):data
            if n == x.v2:
                neighbours.append((x.v1, x))

        #Loop through neighbours
        for neighbour in neighbours:
            if neighbour[0].solved:
                continue
            if neighbour[0] not in N:
                N.add(neighbour[0])
            altDistance = n.dist + neighbour[1].data
            
            if neighbour[0].dist > altDistance:
                neighbour[0].dist = altDistance
                neighbour[0].prev = n
    
    #Reroute your way back 
    nodeptr = finish.prev
    route.append(finish.id)
    while nodeptr != start:
        route.append(nodeptr.id)
        nodeptr = nodeptr.prev
    route.append(start.id)
    return finish.dist, route[::-1] #Reverse the array
            
dist, route = CGraphFindShortestPath(gr, v1, v5)
print("\nThe Results Are: ")
print("Shortest distance = "+str(dist))
print("Shortest route = "+str(route)+"\n\n")

print("====================DGraph====================")
DGraph = dict
gr2 = {1: ["start", {2:7, 3:9, 6:14}],
       2: ["", {1:7, 3:10, 4:15}], 
       3: ["", {1:9, 2:10, 4:11, 6:2}], 
       4: ["", {2:15, 3:11, 5:6}], 
       5: ["goal", {4:6, 6:9}], 
       6: ["", {1:14, 3:2, 5:9}]
      }

def prettyPrint(graph):
    for key in graph:
        print(str(key)+" has neighbours: "+str(graph[key][1]))
        print("\tDist: "+str(graph[key][2][1]))
        print("\tPrev: "+str(graph[key][3][1]))
        print("\tFinished: "+str(graph[key][4][1]))

def getMinDist(graph, N):
    lowestDist = float('inf')
    nodeWithLowestDist = None
    for node in N:
        # print("Is "+str(graph[node][2][1])+" < "+str(lowestDist))
        if graph[node][2][1] < lowestDist:
            nodeWithLowestDist = node
            lowestDist = graph[node][2][1]
    return nodeWithLowestDist

def setDefaultValues2(a):
    for n in a:
        a[n].append(["Dist", float("inf")]) #On index 2
        a[n].append(["Prev", None])         #On index 3
        a[n].append(["Solved", False])      #On index 4

def DGraphFindShortestPath(graph, start, finish):
    setDefaultValues2(graph)
    route = []
    graph[start][2][1] = 0 #Set start on 0
    N = set()
    N.add(start)
    S = set()
    while len(N) != 0:
        n = getMinDist(graph, N)
        S.add(n)
        N.remove(n)
        graph[n][4][1] = True #Solved = true
        if n == finish:
            break
        
        #Add all neighbours to the list
        neighbours = set() #Clear neighbours
        keys = graph[n][1].keys()
        for key in keys:
            neighbours.add(key)

        #Loop through all neighbours  
        for m in neighbours:
            if graph[m][4][1] == True: #Check solved
                continue
            if m not in N: 
                N.add(m)
            altDistance = graph[n][2][1] + graph[n][1][m] #lengte van edge tussen n,m
            if graph[m][2][1] > altDistance:
                graph[m][2][1] = altDistance  #Set new distance
                graph[m][3][1] = n            #Set prevn))
        

    #Reroute your way back 
    nodeptr = graph[finish][3][1]
    # print("Ptr: "+str(nodeptr))
    route.append(finish)
    while nodeptr != start:
        route.append(nodeptr)
        nodeptr = graph[nodeptr][3][1] #Gives an error if the path hasnt been completed
    route.append(start)
    return graph[finish][2][1], route[::-1] #Reverse the array


print(gr2)
print()
dist, route = DGraphFindShortestPath(gr2, 1, 5)
print("\nThe Results Are: ")
print("Shortest distance = "+str(dist))
print("Shortest route = "+str(route)+"\n\n")




====================CGraph====================
V: {1:start, 2:, 3:, 4:, 5:goal, 6:}
E: {(1,6):14, (3,6):2, (2,4):15, (2,3):10, (4,5):6, (1,2):7, (1,3):9, (3,4):11, (5,6):9}


The Results Are: 
Shortest distance = 20
Shortest route = [1, 3, 6, 5]


====================DGraph====================
{1: ['start', {2: 7, 3: 9, 6: 14}], 2: ['', {1: 7, 3: 10, 4: 15}], 3: ['', {1: 9, 2: 10, 4: 11, 6: 2}], 4: ['', {2: 15, 3: 11, 5: 6}], 5: ['goal', {4: 6, 6: 9}], 6: ['', {1: 14, 3: 2, 5: 9}]}


The Results Are: 
Shortest distance = 20
Shortest route = [1, 3, 6, 5]




*Schrijf hier de tekst-antwoorden voor Opgave 1*
CGraph (111.80us) is langzamer dan DGraph(62.00us)

## Opgave 2: Een andere implentatie?

Verzin (evt. samen met je teamleden) nog een andere manier om grafen te implementeren, en schrijf vervolgens weer een bijbehorende implementatie van Dijkstra's algoritme. Is deze beter dan de vorige twee manieren? Toon dit wederom aan door metingen te doen. 

Ter inspiratie: kijk eens naar **adjacency lists** https://en.wikipedia.org/wiki/Adjacency_list en **adjacency matrices** https://en.wikipedia.org/wiki/Adjacency_matrix . Zou je de edges misschien (evt. per vertex) kunnen sorteren op een manier dat Dijkstra's algoritme makkelijker maakt, en zo ja, is dat computationeel de moeite waard?

In [ ]:
#Schrijf hier de code voor opgave 2

*Schrijf hier de tekst-antwoorden voor Opgave 2*

## Ten Slotte

Grafen kun je dus op heel veel verschillende manieren implementeren. Sommige manieren zijn beter dan anderen voor specifieke algoritmes. Let dus altijd goed op hoe je een graaf implementeert als je aan verschillende toepassingen denkt. 

Ook verschilt de implementatie uiteraard als er wel of geen data in vertices en/of edges moeten worden opgeslagen. 

In het tweede grote practicum, zullen we kijken naar een coordination graph. Dit is een graaf waarbij elke vertex een beslisvariabele voorstelt met een beperkt aantal mogelijke beslissingen (acties / waarden). Elke edge representeert welke lokale beloning je krijgt voor de mogelijke beslissingen voor de beslisvariabelen die dezen verbindt. Kijk bij dit practicum eerst eens goed hoe de graaf geïmplementeerd is. Mocht je daar vragen over hebben stel ze dan z.s.m. aan een van de docenten of aan de studentassistent.   